In [ ]:
from IPython.utils import capture
with capture.capture_output() as cap:
  %cd /content
  !git clone https://github.com/comfyanonymous/ComfyUI
  #
  %cd /content/ComfyUI/custom_nodes
  !git clone https://github.com/mit-han-lab/ComfyUI-nunchaku
  !git clone https://github.com/ltdrdata/ComfyUI-Inspire-Pack
  #
  !curl -Lo /content/nunchaku-0.3.2.dev20250701%2Btorch2.6-cp311-cp311-linux_x86_64.whl https://huggingface.co/josemerinom/flux/resolve/main/nunchaku-0.3.2.dev20250701%2Btorch2.6-cp311-cp311-linux_x86_64.whl
  !curl -Lo /content/ComfyUI/models/vae/ae.safetensors https://huggingface.co/josemerinom/flux/resolve/main/ae.safetensors
  !curl -Lo /content/ComfyUI/models/clip/clip_l.safetensors https://huggingface.co/josemerinom/flux/resolve/main/clip_l.safetensors
  !curl -Lo /content/ComfyUI/models/clip/t5xxl_fp8_e4m3fn.safetensors https://huggingface.co/josemerinom/flux/resolve/main/t5xxl_fp8_e4m3fn.safetensors
  !curl -Lo /content/ComfyUI/models/unet/svdq-int4_r32-flux.1-dev.safetensors https://huggingface.co/josemerinom/flux/resolve/main/svdq-int4_r32-flux.1-dev.safetensors
  !curl -Lo /content/ComfyUI/models/unet/svdq-int4_r32-flux.1-kontext-dev.safetensors https://huggingface.co/josemerinom/flux/resolve/main/svdq-int4_r32-flux.1-kontext-dev.safetensors
  #
  !pip install /content/nunchaku-0.3.2.dev20250701%2Btorch2.6-cp311-cp311-linux_x86_64.whl
  !pip install -r /content/ComfyUI/requirements.txt
  !pip install -r /content/ComfyUI/custom_nodes/ComfyUI-Inspire-Pack/requirements.txt
  !pip install -r /content/ComfyUI/custom_nodes/ComfyUI-nunchaku/requirements.txt
  #
print("done")

In [ ]:
%cd /content/ComfyUI
#
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server